In [2]:
import pandas as pd
import sqlite3

url = "https://nycopendata.socrata.com/resource/erm2-nwe9.csv"
data = pd.read_csv(url)

# Crear una conexión a SQLite en memoria
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Cargar el dataframe a SQLite
data.to_sql('nyc_311_service_requests', conn, index=False, if_exists='replace')


1000

In [4]:
# 1. Identificar los Principales Tipos de Quejas
query1 = """
SELECT complaint_type, COUNT(*) AS complaint_count
FROM nyc_311_service_requests
GROUP BY complaint_type
ORDER BY complaint_count DESC
LIMIT 5;
"""

result1 = pd.read_sql_query(query1, conn)
print("Principales Tipos de Quejas:\n", result1)


Principales Tipos de Quejas:
             complaint_type  complaint_count
0          Illegal Parking              195
1      Noise - Residential              178
2  Noise - Street/Sidewalk              127
3         Blocked Driveway               73
4       Noise - Commercial               46


In [5]:
# 2. Analizar Quejas por Distrito
query2 = """
SELECT borough, COUNT(*) AS total_complaints
FROM nyc_311_service_requests
GROUP BY borough
ORDER BY total_complaints DESC;
"""

result2 = pd.read_sql_query(query2, conn)
print("Quejas por Distrito:\n", result2)


Quejas por Distrito:
          borough  total_complaints
0       BROOKLYN               291
1         QUEENS               238
2          BRONX               229
3      MANHATTAN               215
4  STATEN ISLAND                27


In [6]:
# 3. Filtrar Tipos de Quejas de Alto Volumen
query3 = """
SELECT complaint_type, COUNT(*) AS complaint_count
FROM nyc_311_service_requests
GROUP BY complaint_type
HAVING COUNT(*) > 1000
ORDER BY complaint_count DESC;
"""
result3 = pd.read_sql_query(query3, conn)
print("Tipos de Quejas de Alto Volumen:\n", result3)

Tipos de Quejas de Alto Volumen:
 Empty DataFrame
Columns: [complaint_type, complaint_count]
Index: []


In [8]:
# 4. Comparación de Distritos para un Tipo Específico de Queja (ej. 'Noise - Residential')
query4 = """
SELECT borough, COUNT(*) AS noise_complaints
FROM nyc_311_service_requests
WHERE complaint_type = 'Noise - Residential'
GROUP BY borough
ORDER BY noise_complaints DESC;
"""

result4 = pd.read_sql_query(query4, conn)
print("Comparación de Distritos para 'Noise - Residential':\n", result4)


Comparación de Distritos para 'Noise - Residential':
          borough  noise_complaints
0          BRONX                48
1      MANHATTAN                47
2       BROOKLYN                45
3         QUEENS                35
4  STATEN ISLAND                 3


In [9]:
# Cerrar conexión
conn.close()